In [2]:
from skimage import io
from toolbox import convolve, gaussian_kernel, noisy, center_crop_pixel, plot_images
import numpy as np
import torch

In [3]:
img_path = '/Users/luwang/Desktop/projects/personal/psf-estimation/psf-estimation/data/PSF_Estimation/astigmatism/tilted/10/10degres_proche_2/MMStack_Pos0.ome.tif'
img = io.imread(img_path, as_gray=True)

In [4]:
img = img.astype(np.float64)
img /= 65535.0

In [6]:
def generate_syn_img_stack(img, n, zmin, zmax, zfocus, noise_sigma, input_noise, width_coeff):
    z_coeff = 1.7 * width_coeff
    im_size = img.shape[0]
    kernel_size = im_size // 2 + 1
    if kernel_size % 2 == 0: # if even kernel size, make odd
        kernel_size += 1
        
    # generate random noise for the distances from focal point (simulates measurement error)
    noise = np.random.normal(0, noise_sigma, n)
    
    # generate list of kernel z-positions
    z_list = np.linspace(zmin-zfocus+1, zmax-zfocus, n).tolist() # TODO: i don't think the bounds are right
    
    kernels = []
    for z_index, z in enumerate(z_list):
        dist = np.abs(z * z_coeff) + noise[z_index]
        gkernel = gaussian_kernel(kernel_size, dist, dist) * (im_size ** 2) # TODO: why (img_size ** 2)?
        kernels.append(gkernel)
    return kernels, z_list

def convolve_kernels(img, kernels):
    all_images= []
    i = 0
    # uni = np.random.uniform(input_noise // 2, input_noise * 2, len(kernels))
    for kernel in kernels:
        if i % 5 == 0:
            print(i)
        c = convolve(img, kernel, padding='reflect')
        # c = noisy(c, 'gauss', uni[i])
        c = c.clip(0.01,0.95)
        i += 1

        all_images.append(center_crop_pixel(c, img.shape[0]))

    return np.asarray(all_images)

In [7]:
# kernels, z_list = generate_syn_img_stack(img, 51, -30, 30, 0, 0, 0, 1)

In [8]:
# all_images = convolve_kernels(img, kernels)

In [9]:
# io.imshow(img)

In [10]:
# io.imshow(all_images[0])

In [11]:
# io.imshow(all_images[len(all_images)//2 + 1])

In [12]:
# io.imshow(all_images[-1])

In [13]:
# np.save('syn_image_stack.npy', all_images)
# np.save('z_list.npy', np.array(z_list))

In [14]:
def blur_image_stack(image, num_z, min_z_calib = None, max_z_calib = None, z_focus=0, noise_sigma=0.0, input_noise = 0.0, width_coeff = 1.0):
    im_size = image.shape[0]
    #kernels = np.zeros((im_size, num_z, num_z))
    print('Generating a blurred stack from {} to {} with {} images and centered at z={}.'.format(min_z_calib, max_z_calib, num_z, z_focus))
    
    kernels = []
    z_coeff = 1.7*width_coeff
    noise = np.random.normal(0, noise_sigma, num_z)
    kernel_size = im_size // 2 + 1
    if kernel_size % 2 == 0: # if even kernel size, make odd
        kernel_size += 1
    if num_z == 1: # if only 1 sample:
        dist = abs(float(max_z_calib-z_focus) * z_coeff)
        dist += noise[0] # add noise to the distance (this simulates additional gaussian blur)
        kernels.append(gaussian_kernel(kernel_size, fwhmx=dist, fwhmy=dist) * (im_size ** 2))
    else:
        z_list = np.linspace (min_z_calib-z_focus+1, max_z_calib-z_focus, num_z).tolist()
        for z_idx, z in enumerate(z_list):
            if not isinstance(z, float):
                z = z[0]
            dist = np.abs(z*z_coeff)
            dist += noise[z_idx]
            kernels.append(gaussian_kernel(kernel_size, fwhmx=dist, fwhmy=dist) * (im_size ** 2))
    # plot_images(kernels)

    all_images = []
    i = 0
    uni = np.random.uniform(input_noise // 2, input_noise * 2, len(kernels))
    for kernel in kernels:
        c = convolve(image, kernel, padding='reflect')
        c = noisy(c, 'gauss', uni[i])
        c = c.clip(0.01,0.95)
        i +=1

        all_images.append(center_crop_pixel(c,image.shape[0]))
    #plot_images(all_images)
    #plt.show()
    return np.asarray(all_images), np.linspace(min_z_calib, max_z_calib, num_z)


In [15]:
# x = np.array(blur_image_stack(img, 11, -5, 5, 0, 0, 0, 1))